# Basic EDA

## Requirements

In [ ]:
import pandas as pd
import pycountry_convert as pc
import ppscore as pps

import matplotlib.pyplot as plt
import seaborn as sns

from local_nbutils import CFG, plt_savefig

## First Confrontation with the Data

We read the data from file into a pandas data frame and create a copy that will incorporate our manipulations.

In [ ]:
df = pd.read_csv(CFG["TRAIN_DATA_PATH"])

The usual initial inspection commands:

In [ ]:
print(df.head())
print(df.shape)
print(df.isnull().sum())
print(df.dtypes)

df.columns

Meaning of column names (according to <https://zindi.africa/competitions/flight-delay-prediction-challenge/data>):

Present in the data:

| Column | Description |
| --- | --- |
| ID | Unique identifier for the flight |
| DATOP | Date of flight |
| FLTID | Flight number |
| DEPSTN | Departure point (station/airport) |
| ARRSTN | Arrival point (station/airport) |
| STD | Scheduled Time of Departure |
| STA | Scheduled Time of Arrival |
| STATUS | Flight status (e.g., delayed, canceled) |
| AC | Aircraft code |
| target | Flight delay (in minutes) |


Not present in the data (although claimed on the referenced web page):

| Column | Description |
| --- | --- |
| ETD | Expected Time departure |
| ETA | Expected Time arrival |
| ATD | Actual Time of Departure |
| ATA | Actual Time of arrival |
| DELAY1 | Delay code 1 |
| DUR1 | Delay time 1 |
| DELAY2 | Delay code 2 |
| DUR2 | Delay time 2 |
| DELAY3 | Delay code 3 |
| DUR3 | Delay time 3 |
| DELAY4 | Delay code 4 |
| DUR4 | Delay time 4 |

### Status Column

In [ ]:
# Sorry for "statuses" ...
statuses = df["STATUS"].unique()

print("All Statuses:")
for status in statuses:
    print(f"  {status}")
    print(f"    Number of entries: {df[df['STATUS'] == status].shape[0]}")
    print(f"    Mean: {df[df['STATUS'] == status]['target'].mean()}")
    print(f"    Median: {df[df['STATUS'] == status]['target'].median()}")


| Code | Name | Description |
| --- | --- | --- |
| ATA | Actual Time Arrival| Flights that successfully landed at their destination |
| DEP  | Departed | Flights that departed but may not have completed their journey |
| RTR  | Returned | Flights that took off but returned to the departure airport due to issues |
| SCH  | Scheduled | Flights listed in the schedule, no delay data applicable |
| DEL  | Canceled | Flights that were canceled, treated as permanent delays |

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(15, 10))
axes = axes.flatten()

for idx, status in enumerate(statuses):
    ax = axes[idx]
    df[df["STATUS"] == status]["target"].hist(
        bins=50,
        log=False,
        ax=ax
    )
    ax.set_title(status)
    ax.set_xlabel("Delay")
    ax.set_ylabel("Frequency")

plt.tight_layout()

plt_savefig("delay-to-sum-flight-histograms")
plt.show()

The interpretation of DEP remains a bit obscure ... in a first approximation, we drop it. Further, it is hard to measure the delay of a DEL flight (a possibility for regular flights would be to take the duration between the DEL flight and the next flight that indeed arrives plus the delay of that flight). But we also decide us simply for dropping.

In [ ]:
df = df[~df["STATUS"].isin(["DEP", "DEL"])]

### Airport Columns

We introduce columns reducing the airports of departure and destination to its country, respectively.

In [ ]:
airports = pd.read_csv(CFG["AIRPORTS_DATA_PATH"])
airports = airports[["iata_code", "iso_country"]]
airports = airports.dropna()

In [ ]:
df_bkp = df.copy()

In [ ]:
# TODO Make this cell idempotent

df = df_bkp.merge(
    airports[["iata_code", "iso_country"]],
    left_on="DEPSTN",
    right_on="iata_code",
    how="left",
)
df.drop(columns="iata_code", inplace=True)
df.rename(columns={"iso_country": "country_dep"}, inplace=True)

df = df.merge(
    airports[["iata_code", "iso_country"]],
    left_on="ARRSTN",
    right_on="iata_code",
    how="left",
)
df.drop(columns="iata_code", inplace=True)
df.rename(columns={"iso_country": "country_arr"}, inplace=True)

df["country_arr"].shape

df.loc[df["DEPSTN"] == "SXF", "country_dep"] = "DE"
df.loc[df["ARRSTN"] == "SXF", "country_arr"] = "DE"

For converting the iso codes to continent codes, we use the functionality provided by the module `pycountry_convert`.

In [ ]:
def iso_to_continent(iso: str) -> None | str:
    try:
        continent_code = pc.country_alpha2_to_continent_code(iso)
        return pc.convert_continent_code_to_continent_name(continent_code)
    except:
        return None


df["continent_dep"] = df["country_dep"].apply(iso_to_continent)
df["continent_arr"] = df["country_arr"].apply(iso_to_continent)

### Dating issues

The data set contains several columns with date semantics. Let us convert them to the appropriate dtype.

In [ ]:
df["DATOP"] = pd.to_datetime(df["DATOP"], format="%Y-%m-%d")
df["STD"] = pd.to_datetime(df["STD"], format="%Y-%m-%d %H:%M:%S")
df["STA"] = pd.to_datetime(df["STA"], format="%Y-%m-%d %H.%M.%S")

Now, we can introduce a bunch of further useful date and time related columns:

In [ ]:
df["DATOP_year"] = df["DATOP"].dt.year
df["DATOP_month"] = df["DATOP"].dt.month
df["DATOP_day"] = df["DATOP"].dt.dayofweek + 1

def map_hour_to_period(hour: int) -> str:
    if 6 <= hour < 12:
        return "morning"
    elif 12 <= hour < 18:
        return "day"
    elif 18 <= hour < 24:
        return "evening"
    else:
        return "night"


df["STD_hour"] = df["STD"].dt.hour
df["STD_period"] = df["STD_hour"].apply(map_hour_to_period)

df["flight_time"] = (df["STA"] - df["STD"]).dt.total_seconds() / 60

Which years are actually present?

In [ ]:
DATOP_years = df["DATOP_year"].unique()
DATOP_years

So the data are from the years 2016, 2017, 2018. 

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=len(DATOP_years), figsize=(16, 5), sharey=True)

for idx, year in enumerate(DATOP_years):
    # Filter the DataFrame for the specific year
    df_year = df[df["DATOP_year"] == year]
    
    # Plot the histogram on the respective subplot
    axes[idx].hist(df_year["DATOP_month"], bins=range(1, 14), alpha=0.8, color="blue")
    axes[idx].set_title(f"Flight Distribution for {year}")
    axes[idx].set_xlabel("Month")
    axes[idx].set_xticks(range(1, 13))  # Set x-axis ticks for months
    axes[idx].set_ylabel("Number of Flights")

plt.tight_layout()

plt_savefig("month-to-sum-flight-by-year_hist.png")

plt.show()

In each year, we find a suspicious months in which the sum of flights is significantly less than in the others. Looking at the provided test data set one sees that the majority of flights for the affected months can be found there (sic!) ... We drop these months completely: 

In [ ]:
df = df[~((df["DATOP_month"] == 5) & (df["DATOP_year"] == 2016))]
df = df[~((df["DATOP_month"] == 2) & (df["DATOP_year"] == 2017))]
df = df[~((df["DATOP_month"] == 9) & (df["DATOP_year"] == 2018))]

df = df[df["DEPSTN"] != df["ARRSTN"]]

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(df["flight_time"], df["target"], color="blue")
plt.xlabel("Flight Time")
plt.ylabel("Delay")
plt.xlim(1,3000)
plt.ylim(1,3000)
plt.title("Scatter Plot of X Column vs Y Column")
plt.show()

In [ ]:
num_years = len(DATOP_years)
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 5))
axes = axes.flatten()

for i, year in enumerate(DATOP_years):
    ax = axes[i]  # Access subplot
    df_year = df[df["DATOP_year"] == year]
    daily_avg = df_year.groupby("DATOP_day")["target"].mean().reset_index()
    
    # Plot bar plot on the current subplot
    ax.bar(daily_avg["DATOP_day"], daily_avg["target"], color="blue", alpha=0.7)
    
    # Customize the plot
    ax.set_title(f"Average Delay by Day of Week for {year}", fontsize=12)
    ax.set_xlabel("Month", fontsize=10)
    ax.set_ylabel("Delay", fontsize=10)
    ax.set_xticks(range(1, 8)) 
    ax.grid(axis="y", linestyle="--", alpha=0.7)

# Adjust layout and save the figure
plt.tight_layout()
plt_savefig("month-to-avg-delay-by-year_hist") 
plt.show()


## Final Look at the Brushed Data


In [ ]:
df.head()

A scatterplot gives a feeling for single and pairwise distributions:

In [ ]:
sns.pairplot(df)

plt_savefig("each-vs-each-wrt-distribution_scatterplot")
plt.show()

Inspecting the correlation matrix is never a bad idea:

In [ ]:
correlation_matrix = df.corr()

# Plot the correlation matrix as a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(
    correlation_matrix,
    annot=True,
    fmt=".2f",
)
plt.title("Correlation Matrix")

plt_savefig("each-vs-each-wrt-correlation_heatmap")
plt.show()

But as we have so many categorical features, it makes sense to compute the power predictive score (pps) matrix:

In [ ]:
cols = [
    col 
    for col in df.columns
    if not col.startswith("DATOP_") and col not in ["ID"]
]

df_tmp = df[cols]

pp_scores = pps.matrix(df_tmp)[["x", "y", "ppscore"]].pivot(
    columns="x", index="y", values="ppscore"
)

pp_scores = pp_scores.round(2)

plt.figure(figsize=(12, 8))

sns.heatmap(
    pp_scores,
    vmin=0,
    vmax=1,
    # cmap="Reds",
    linewidths=0.5,
    annot=True,
)

plt_savefig("each-vs-each-wrt-pp-score_heatmap")

plt.plot()

In [ ]:
df.to_pickle(CFG["PROCESSED_DATA_PATH"])